In [26]:
# The goal so far is to identify dates of prescription and then return the 3 month window from the date of prescription to date of opioid related harm

In [2]:
import pandas as pd

In [1]:
!ls 

Data Preparation.ipynb	filter_list_no_star_linecode.txt  README.md
exclude_cancer.sh	New_data_prep.ipynb
filter_list_no_star	non_cancer_op_prescribed.csv


In [28]:
# prescribed = pd.read_csv('../datasets/cms_processed_full/prescription_final19.csv', index_col =0)
# inp = pd.read_csv('../datasets/cms_processed_full/inpatient_final19.csv', index_col =0)
# outp = pd.read_csv('../datasets/cms_processed_full/outpatient_final19.csv', index_col =0)
prescribed = pd.read_csv('../datasets/cms_processed_full/prescription_final1.csv', index_col =0)
inp = pd.read_csv('../datasets/cms_processed_full/inpatient_final1.csv', index_col =0)
outp = pd.read_csv('../datasets/cms_processed_full/outpatient_final1.csv', index_col =0)

In [10]:
presc = pd.read_csv('../datasets/cms_download/DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.csv')
inp = pd.read_csv('../datasets/cms_download/DE1_0_2008_to_2010_Inpatient_Claims_Sample_1.csv')
outp = pd.read_csv('../datasets/cms_download/DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv')

/tmp/ipykernel_6470/656714403.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  presc = pd.read_csv('../datasets/cms_download/DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.csv')
/tmp/ipykernel_6470/656714403.py:3: DtypeWarning: Columns (21,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  outp = pd.read_csv('../datasets/cms_download/DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv')


In [5]:
import pickle
with open('filter_list_no_star', 'rb') as f:
    filter_list_no_star = pickle.load(f)

In [11]:
first_prescribed = presc.query('PROD_SRVC_ID == @filter_list_no_star').reset_index(drop=True)

In [12]:
first_prescribed

,DESYNPUF_ID,PDE_ID,SRVC_DT,PROD_SRVC_ID,QTY_DSPNSD_NUM,DAYS_SUPLY_NUM,PTNT_PAY_AMT,TOT_RX_CST_AMT
0,26D48234B1586158,233774488789326,20091217,00143100552,100.0,20,0.0,20.0
1,26D48234B1586158,233464492513148,20091220,63874020201,60.0,20,0.0,30.0
2,26D5584EEDAAE238,233704488950222,20090114,68115039799,20.0,80,0.0,20.0
3,26D5584EEDAAE238,233834489985005,20090725,33261005830,60.0,10,0.0,90.0
4,26D59CF00FF689CA,233144490974904,20091209,68462019301,120.0,10,10.0,10.0
...,...,...,...,...,...,...,...,...
28049,FDEBD2245BE84267,233024490882465,20091119,55289064830,60.0,20,10.0,170.0
28050,FDEDCC5E1626117B,233404489837851,20080509,63874020305,50.0,30,0.0,10.0
28051,FDEDCC5E1626117B,233934488876345,20080525,00591321372,30.0,30,0.0,70.0
28052,FDEDCC5E1626117B,233134492585708,20100415,58016007705,120.0,30,10.0,20.0


In [15]:
import numpy as np 
fp = first_prescribed[['SRVC_DT', 'DESYNPUF_ID']]
# fp.min(axis='columns')
fp = fp.groupby('DESYNPUF_ID').agg({'SRVC_DT':np.min}).reset_index()
fp

,DESYNPUF_ID,SRVC_DT
0,26D48234B1586158,20091217
1,26D5584EEDAAE238,20090114
2,26D59CF00FF689CA,20091209
3,26D5A2B67FF2F770,20080630
4,26D7F5CD9BCC7ECB,20101027
...,...,...
11466,FDE61CA99097DABA,20080928
11467,FDE9D86C44F4A51B,20080715
11468,FDEBD2245BE84267,20081120
11469,FDEDCC5E1626117B,20080509


In [7]:
unifp = pd.concat([inp,outp]).reset_index(drop = True)


,PDE_ID,SRVC_DT,PROD_SRVC_ID,QTY_DSPNSD_NUM,DAYS_SUPLY_NUM,PTNT_PAY_AMT,TOT_RX_CST_AMT
0,233774488789326,20091217,00143100552,100.0,20,0.0,20.0
1,233464492513148,20091220,63874020201,60.0,20,0.0,30.0
2,233704488950222,20090114,68115039799,20.0,80,0.0,20.0
3,233834489985005,20090725,33261005830,60.0,10,0.0,90.0
4,233144490974904,20091209,68462019301,120.0,10,10.0,10.0
...,...,...,...,...,...,...,...
28049,233024490882465,20091119,55289064830,60.0,20,10.0,170.0
28050,233404489837851,20080509,63874020305,50.0,30,0.0,10.0
28051,233934488876345,20080525,00591321372,30.0,30,0.0,70.0
28052,233134492585708,20100415,58016007705,120.0,30,10.0,20.0


In [55]:
unifp = pd.concat([inp,outp]).reset_index(drop = True)
set(prescribed.DESYNPUF_ID)
prescribed

,DESYNPUF_ID,PDE_ID,SRVC_DT,PROD_SRVC_ID,QTY_DSPNSD_NUM,DAYS_SUPLY_NUM,PTNT_PAY_AMT,TOT_RX_CST_AMT
0,3951A55F8A5682D3,233854493404544,20080218,61392001445,30.0,30,0.0,20.0
1,3951A55F8A5682D3,233704489018890,20080224,12280012660,30.0,30,60.0,80.0
2,3951A55F8A5682D3,233204493646195,20080229,904100716,30.0,90,0.0,0.0
3,3951A55F8A5682D3,233524491860147,20080409,67544023630,100.0,60,10.0,20.0
4,3951A55F8A5682D3,233424492895725,20080420,58016058503,30.0,90,0.0,40.0
...,...,...,...,...,...,...,...,...
1411,FC0D6ECA50586A4B,233544489310586,20100301,65427078550,60.0,30,80.0,210.0
1412,FC0D6ECA50586A4B,233804490653269,20100301,19980022805,10.0,80,0.0,10.0
1413,FC0D6ECA50586A4B,233174490875013,20100302,51079039410,30.0,10,10.0,0.0
1414,FC0D6ECA50586A4B,233544492894182,20100305,61392042051,60.0,90,0.0,10.0


In [50]:
unifp
len(outp),len(inp), len(unifp)
outp.shape, inp.shape, unifp.shape, prescribed.shape


((123, 61), (26, 61), (149, 61), (1416, 8))

In [31]:
import pickle
with open('filter_list_no_star', 'rb') as f:
    filter_list_no_star = pickle.load(f)

In [52]:
o_presc = pd.read_csv('../datasets/cms_download/DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.csv')
o_presc.query('PROD_SRVC_ID == @filter_list_no_star')

/tmp/ipykernel_27143/3829498507.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  o_presc = pd.read_csv('../datasets/cms_download/DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_1.csv')


,DESYNPUF_ID,PDE_ID,SRVC_DT,PROD_SRVC_ID,QTY_DSPNSD_NUM,DAYS_SUPLY_NUM,PTNT_PAY_AMT,TOT_RX_CST_AMT
852015,26D48234B1586158,233774488789326,20091217,00143100552,100.0,20,0.0,20.0
852016,26D48234B1586158,233464492513148,20091220,63874020201,60.0,20,0.0,30.0
852107,26D5584EEDAAE238,233704488950222,20090114,68115039799,20.0,80,0.0,20.0
852132,26D5584EEDAAE238,233834489985005,20090725,33261005830,60.0,10,0.0,90.0
852252,26D59CF00FF689CA,233144490974904,20091209,68462019301,120.0,10,10.0,10.0
...,...,...,...,...,...,...,...,...
5504701,FDEBD2245BE84267,233024490882465,20091119,55289064830,60.0,20,10.0,170.0
5504812,FDEDCC5E1626117B,233404489837851,20080509,63874020305,50.0,30,0.0,10.0
5504814,FDEDCC5E1626117B,233934488876345,20080525,00591321372,30.0,30,0.0,70.0
5504902,FDEDCC5E1626117B,233134492585708,20100415,58016007705,120.0,30,10.0,20.0


In [32]:
first_prescribed = prescribed.query('PROD_SRVC_ID == @filter_list_no_star').reset_index(drop=True)

In [33]:
first_prescribed


,DESYNPUF_ID,PDE_ID,SRVC_DT,PROD_SRVC_ID,QTY_DSPNSD_NUM,DAYS_SUPLY_NUM,PTNT_PAY_AMT,TOT_RX_CST_AMT


In [34]:
prescribed.query('SRVC_DT < 20080818 & DESYNPUF_ID == "06735B95C3BE024E"').reset_index(drop=True)

,DESYNPUF_ID,PDE_ID,SRVC_DT,PROD_SRVC_ID,QTY_DSPNSD_NUM,DAYS_SUPLY_NUM,PTNT_PAY_AMT,TOT_RX_CST_AMT


In [35]:
import numpy as np
fp = first_prescribed[['SRVC_DT', 'DESYNPUF_ID']]
# fp.min(axis='columns')
fp = fp.groupby('DESYNPUF_ID').agg({'SRVC_DT':np.min}).reset_index()
fp

,DESYNPUF_ID,SRVC_DT


In [36]:
from datetime import datetime, timedelta
def change_date(date,delta):
    today = datetime.strptime(str(date), '%Y%M%d') 
    prior = today + timedelta(days = delta)
    return prior.strftime('%Y%M%d')




In [53]:
b_presc = pd.DataFrame(columns= first_prescribed.columns)
b_unif  = pd.DataFrame(columns = unifp.columns)
a_presc = pd.DataFrame(columns= first_prescribed.columns)
a_unif  = pd.DataFrame(columns = unifp.columns)
for index, row in fp.iterrows():
    # print( row)
    date = row['SRVC_DT']
    three_months_before_date = change_date(date, -900)
    three_months_after_date = change_date(date, 900)
    id = row['DESYNPUF_ID']

    b_presc = pd.concat([b_presc, prescribed.query('%s> SRVC_DT >%s  & DESYNPUF_ID == "%s"'%(date, three_months_before_date, id))])
    b_unif = pd.concat([b_unif, unifp.query('%s> CLM_FROM_DT >%s & DESYNPUF_ID == "%s"'%(date, three_months_before_date, id))])
    
    a_presc = pd.concat([a_presc, prescribed.query('%s< SRVC_DT <%s & DESYNPUF_ID == "%s"'%(date, three_months_after_date, id))])
    a_unif = pd.concat([a_unif, unifp.query('%s< CLM_FROM_DT <%s & DESYNPUF_ID == "%s"'%(date, three_months_after_date, id))])

    a_presc  
b_presc = b_presc.reset_index(drop=True)
b_unif = b_unif.reset_index(drop = True)

a_presc = a_presc.reset_index(drop=True)
a_unif = a_unif.reset_index(drop = True)
a_unif




,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,ICD9_DGNS_CD_3,ICD9_DGNS_CD_4,...,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45


In [38]:
custom_columns_inp_outp = ['DESYNPUF_ID', 'CLM_ID', 'SEGMENT', 'CLM_FROM_DT', 'CLM_THRU_DT','PRVDR_NUM','ICD9_DGNS_CD_1', 'ICD9_DGNS_CD_2', 'ICD9_DGNS_CD_3', 'ICD9_DGNS_CD_4','ICD9_DGNS_CD_5', 'ICD9_DGNS_CD_6', 'ICD9_DGNS_CD_7', 'ICD9_DGNS_CD_8','ICD9_DGNS_CD_9', 'ICD9_DGNS_CD_10','HCPCS_CD_1','HCPCS_CD_2', 'HCPCS_CD_3', 'HCPCS_CD_4', 'HCPCS_CD_5', 'HCPCS_CD_6','HCPCS_CD_7', 'HCPCS_CD_8', 'HCPCS_CD_9', 'HCPCS_CD_10', 'HCPCS_CD_11','HCPCS_CD_12', 'HCPCS_CD_13', 'HCPCS_CD_14', 'HCPCS_CD_15','HCPCS_CD_16', 'HCPCS_CD_17', 'HCPCS_CD_18', 'HCPCS_CD_19','HCPCS_CD_20', 'HCPCS_CD_21', 'HCPCS_CD_22', 'HCPCS_CD_23','HCPCS_CD_24', 'HCPCS_CD_25', 'HCPCS_CD_26', 'HCPCS_CD_27','HCPCS_CD_28', 'HCPCS_CD_29', 'HCPCS_CD_30', 'HCPCS_CD_31','HCPCS_CD_32', 'HCPCS_CD_33', 'HCPCS_CD_34', 'HCPCS_CD_35','HCPCS_CD_36', 'HCPCS_CD_37', 'HCPCS_CD_38', 'HCPCS_CD_39','HCPCS_CD_40', 'HCPCS_CD_41', 'HCPCS_CD_42', 'HCPCS_CD_43','HCPCS_CD_44', 'HCPCS_CD_45']

In [39]:
unifp.query('CLM_FROM_DT < 20081101')

,DESYNPUF_ID,CLM_ID,SEGMENT,CLM_FROM_DT,CLM_THRU_DT,PRVDR_NUM,ICD9_DGNS_CD_1,ICD9_DGNS_CD_2,ICD9_DGNS_CD_3,ICD9_DGNS_CD_4,...,HCPCS_CD_36,HCPCS_CD_37,HCPCS_CD_38,HCPCS_CD_39,HCPCS_CD_40,HCPCS_CD_41,HCPCS_CD_42,HCPCS_CD_43,HCPCS_CD_44,HCPCS_CD_45
6,8E604CFCC55733DF,196651176987958,1,20080928.0,20081004.0,3600NG,29570,V1005,30391,78039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,940AE53D99A77425,196681176966935,1,20080228.0,20080307.0,0420UG,34590,2859,7802,V1046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,940AE53D99A77425,196201176982738,1,20080628.0,20080630.0,0401KM,41401,2724,5859,53081,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,CD083800DF63BBA6,196761176989964,1,20080418.0,20080423.0,15016B,99529,3384,96509,V148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,CD083800DF63BBA6,196271176989888,1,20080710.0,20080807.0,1501SP,V5878,70703,2639,2724,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,CD083800DF63BBA6,196891176965595,1,20080922.0,20080925.0,1501SP,42731,3004,7837,E9240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,E52284BB1AAA50DE,196321176989407,1,20080306.0,20080310.0,4300HR,96509,07030,4280,25000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,E52284BB1AAA50DE,196841176993315,1,20080608.0,20080611.0,2400QS,29634,V6284,7999,27651,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,8E604CFCC55733DF,542702281384362,1,20080211.0,20080302.0,3626MJ,V5842,4280,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,8E604CFCC55733DF,542612281360253,1,20080303.0,20080304.0,3600PV,9648,96509,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
result.drop_duplicates()

NameError: name 'result' is not defined

In [ ]:
# len(result.DESYNPUF_ID.unique())
len(prescribed.DESYNPUF_ID.unique())


31

In [ ]:
import os
def run_cmd_cancer_patients(n):
    cmd = '''
    sed -n "1w cancer_patients%s.csv"  ./unifp_op%s.csv
    grep -E ',140..,|,141..,|,142..,|,143..,|,144..,|,145..,|,146..,|,147..,|,148..,|,149..,|,150..,|,151..,|,152..,|,153..,|,154..,|,155..,|,156..,|,156..,|,157..,|,158..,|,159..,|,160..,|,161..,|,162..,|,163..,|,164..,|,165..,|,170..,|,171..,|,172..,|,174..,|,1750.,|,1759.,|,176..,|,179,|,180..,|,181,|,182..,|,183..,|,184..,|,185,|,1860.,|,1869.,|,187..,|,188..,|,189..,|,190..,|,191..,|,192..,|,193,|,194..,|,195..,|,196..,|,197..,|,198..,|,19881,|,19882,|,19889,|,199..,|,2000.,|,2001.,|,2002.,|,2003.,|,2004.,|,2005.,|,2006.,|,2007.,|,2008.,|,2010.,|,2011.,|,2012.,|,2014.,|,2015.,|,2016.,|,2017.,|,2019.,|,2020.,|,2021.,|,2022.,|,2023.,|,2024.,|,2025.,|,2026.,|,2027.,|,2028.,|,2029.,|,2030.,|,2031.,|,2038.,|,2040.,|,2041.,|,2042.,|,2048.,|,2049.,|,2050.,|,2051.,|,2052.,|,2053.,|,2058.,|,2059.,|,2060.,|,2061.,|,2062.,|,2068.,|,2069.,|,2070.,|,2071.,|,2072.,|,2078.,|,2080.,|,2081.,|,2082.,|,2088.,|,2089.,|,2090.,|,2091.,|,2092.,|,2093.,|,2097.,' ./unifp_op%s.csv >> cancer_patients%s.csv
    ''' %(n,n,n,n)
    os.system(cmd)
    os.system('echo "run_cmd_cancer_patients: accessed unifp_op%s to create cancer_patients%s.csv"'%(n,n))



In [ ]:
def run_cmd_opioid_harm_patients(n):
    # following icd9 codes correspond to opioid related harm 
    # source: supplementary appendix for developing prediction algorithm using Pennsylvania Medicaid data, jonassan et al.
    cmd = '''
    sed -n "1w opioid_harm_patients%s.csv"  ./op_prescribed_no_cancer_df%s.csv
    grep -E ',96500,|,96501,|,96502,|,96509,|,E8500,|,E8501,|,E8502,|,E9350,|,E9351,|,E9352,|,3055,|,3040,|,3047,|,E9350,|,E9351,|,E9352,|,E9401,|,9701,' ./op_prescribed_no_cancer_df%s.csv >> opioid_harm_patients%s.csv
    '''%(n,n,n,n)
    os.system(cmd)
    os.system('echo "run_cmd_opioid_harm_patients: accessed op_prescribed_no_cancer_df%s.csv to create opioid_harm_patients%s.csv"'%(n,n))
# run_cmd_opioid_harm_patients(1)

In [ ]:
from os.path import exists
def modularize(sample_number):
    summ_loc = '../cms_download/DE1_0_2008_Beneficiary_Summary_File_Sample_'+str(sample_number)+'.csv'
    inp_loc = '../cms_download/DE1_0_2008_to_2010_Inpatient_Claims_Sample_'+str(sample_number)+'.csv'
    outp_loc = '../cms_download/DE1_0_2008_to_2010_Outpatient_Claims_Sample_'+str(sample_number)+'.csv'
    presc_loc = '../cms_download/DE1_0_2008_to_2010_Prescription_Drug_Events_Sample_'+str(sample_number)+'.csv'
    
    summ, presc = [pd.read_csv(x) for x in [summ_loc, presc_loc]]
    inp, outp =[pd.read_csv(x, usecols = custom_columns_inp_outp) for x in [inp_loc, outp_loc]]

#     summ, presc = [pd.read_csv(x, nrows = 1000000) for x in [summ_loc, presc_loc]]
#     inp, outp =[pd.read_csv(x, usecols = custom_columns_inp_outp, nrows =10) for x in [inp_loc, outp_loc]]
    unifp = inp.append(outp).reset_index(drop= True)

    # plan:
    # op_first_prescribed = a list of users first prescribed opioids and bzds.
    # presc.query -> aggregate np.min to get date, user pair of first prescription drug event 
    
    
#     op_prescribed = a list of users that are prescribed opioids and BZDs
    op_prescribed = list(set(presc.query('PROD_SRVC_ID == @filter_list_no_star').reset_index(drop=True).DESYNPUF_ID))
    


    unifp_op = unifp.query('DESYNPUF_ID == @op_prescribed').reset_index(drop=True)
    
    unifp_op.to_csv('unifp_op' + str(sample_number) + '.csv',index=False)
    run_cmd_cancer_patients(sample_number)
#     returns cancer_patients%s %(sample_number)
    
    cancer_df = pd.read_csv('./cancer_patients'+str(sample_number)+'.csv')
    
    cancer_patients = list(set(cancer_df.DESYNPUF_ID))
    op_prescribed_no_cancer = [x for x in op_prescribed if x not in cancer_patients]
    
    op_prescribed_no_cancer_df = unifp_op.query('DESYNPUF_ID != @cancer_patients').reset_index(drop=True)
    op_prescribed_no_cancer_df.to_csv('op_prescribed_no_cancer_df'+str(sample_number)+'.csv', index=False)
    run_cmd_opioid_harm_patients(sample_number)
    
    opioid_harm_df = pd.read_csv('./opioid_harm_patients'+str(sample_number)+'.csv')
    opioid_harm_list = list(set(opioid_harm_df.DESYNPUF_ID))
    
    print('len(op_prescribed),len(cancer_patients), len(op_prescribed_no_cancer)')
    print(len(op_prescribed),len(cancer_patients), len(op_prescribed_no_cancer))
    print(len(opioid_harm_df))

    print('querying harmlist onto init loaded dfs (summary, presc., inp., outp.)')
    # summ_final, presc_final, inp_final, outp_final = [x.query('DESYNPUF_ID == @opioid_harm_list').reset_index(drop=True) for x in [summ,presc,inp,outp]]
    summ_final= summ.query('DESYNPUF_ID == @opioid_harm_list').reset_index(drop=True)
    presc_final= presc.query('DESYNPUF_ID == @opioid_harm_list').reset_index(drop=True)
    inp_final= inp.query('DESYNPUF_ID == @opioid_harm_list').reset_index(drop=True)   
    outp_final= outp.query('DESYNPUF_ID == @opioid_harm_list').reset_index(drop=True)


    print('writing to ../cms_processed_full/') 
    summ_final.to_csv('../cms_processed_full/summary_final'+ str(sample_number)+'.csv', index= False)
    presc_final.to_csv('../cms_processed_full/prescription_final'+ str(sample_number)+'.csv', index= False)
    inp_final.to_csv('../cms_processed_full/inpatient_final'+ str(sample_number)+'.csv', index= False)
    outp_final.to_csv('../cms_processed_full/outpatient_final'+ str(sample_number)+'.csv', index= False)
    return opioid_harm_df


modularize(2)
# for i in range(1,21):
#     print(len(modularize(i)))